In [2]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com' #防止无法下载的问题


import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)


#设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"当前使用的是cuda/cpu?: {device}")

def load_model_and_tokenizer(): #加载模型和分词器
    model_name = "Qwen/Qwen1.5-7B"
    
    #加载tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir='./myModels'
    )
    if tokenizer.pad_token is None: #pad_token是用来填充较短的序列的
        tokenizer.pad_token = tokenizer.eos_token #结束token
    
    #加载模型
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto", #自动将模型分布到设备上
        load_in_4bit=True,  #使用4位量化模型压缩技术，减少模型内存占用，稍微降低性能
        trust_remote_code=True, #允许从远程模型代码
        cache_dir='./myModels', #下载的模型储存的位置
    )
    
    return model, tokenizer

当前使用的是cuda/cpu?: cuda


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

def setup_LoRA(model): #初始化LoRA
    
    model = prepare_model_for_kbit_training(model) #准备模型用于k-bit训练
    
    
    lora_config = LoraConfig( #LoRA配置
        r=8, #LoRA秩,很重要的一个参数。r越小训练越快，但拟合能力会下降
        lora_alpha=16, #LoRA alpha参数，用于对ΔW进行缩放,通常为LoRA秩的两倍
        target_modules=["q_proj", "v_proj", "gate_proj", "down_proj"], #指定在哪些层应用LoRA，分别是注意力机制的Query Projection和Value Projection，前馈神经网络FFN的Gate Projection和Down Projection
        lora_dropout=0.1, #正则化，dropout率，防止过拟合，
        bias="none", #不训练偏置参数
        task_type="QUESTION_ANS", #指定任务类型，这里是问答任务
    )
    
    model = get_peft_model(model, lora_config) #将LoRA适配器应用到原来的模型上

    print('-----------------下面是可训练的参数-----------------')
    model.print_trainable_parameters() #打印可训练的参数信息
    print('-----------------上面是可训练的参数-----------------')
    
    return model

In [ ]:
if __name__ == "__main__":
    #加载模型和tokenizer
    print("正在加载模型和分词器")
    model, tokenizer = load_model_and_tokenizer()
    print("加载完成")

    #初始化LoRA
    print("正在加载模型和分词器")
    model = setup_LoRA(model)
    print("初始化完成")

正在加载模型和分词器


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:37<00:00,  9.25s/it]
